# Agent for Project Progress Report

In [1]:
# Warning control
import warnings
warnings.filterwarnings('ignore')


In [2]:
import os
from utils import get_openai_api_key
from dotenv import load_dotenv
import google.generativeai as genai
from langchain_google_genai import ChatGoogleGenerativeAI
from crewai import Agent, Task, Crew
import yaml


In [14]:
load_dotenv()
openai_api_key = get_openai_api_key()


In [25]:
os.environ['OPENAI_MODEL_NAME'] = 'gpt-3.5-turbo-0125'


In [19]:
# Define file paths for YAML configurations
files = {
    'agents': 'config/pr_agents.yaml',
    'tasks': 'config/pr_tasks.yaml'
}

# Load configurations from YAML files
configs = {}
for config_type, file_path in files.items():
    with open(file_path, 'r') as file:
        configs[config_type] = yaml.safe_load(file)

# Assign loaded configurations to specific variables
agents_config = configs['agents']
tasks_config = configs['tasks']


In [30]:

import requests

url = f"{os.getenv('DLAI_TRELLO_BASE_URL', 'https://api.trello.com')}/1/boards/{os.environ['TRELLO_BOARD_ID']}/cards"

query = {
    'key': os.environ['TRELLO_API_KEY'],
    'token':os.environ['TRELLO_API_TOKEN'],
    'fields': 'name,idList,due,dateLastActivity,labels',
    'attachments': 'true',
    'actions': 'commentCard'
}

response = requests.get(url, params=query)
response


<Response [401]>

In [22]:
from crewai.tools import BaseTool 
import requests
import json


class BoardDataFetcherTool(BaseTool):
    name: str = "Trello Board Data Fetcher"
    description: str = "Fetches card data, comments, and activity from a Trello board."

    api_key: str = os.environ['TRELLO_API_KEY']
    api_token: str = os.environ['TRELLO_API_TOKEN']
    board_id: str = os.environ['TRELLO_BOARD_ID']

    def _run(self) -> dict:
        """
        Fetch all cards from the specified Trello board.
        """
        url = f"{os.getenv('DLAI_TRELLO_BASE_URL', 'https://api.trello.com')}/1/boards/{self.board_id}/cards"

        query = {
            'key': self.api_key,
            'token': self.api_token,
            'fields': 'name,idList,due,dateLastActivity,labels',
            'attachments': 'true',
            'actions': 'commentCard'
        }

        response = requests.get(url, params=query)

        if response.status_code == 200:
            return response.json()
        else:
            # Fallback in case of timeouts or other issues
            return json.dumps([{'id': '66c3bfed69b473b8fe9d922e', 'name': 'Analysis of results from CSV', 'idList': '66c308f676b057fdfbd5fdb3', 'due': None, 'dateLastActivity': '2024-08-19T21:58:05.062Z', 'labels': [], 'attachments': [], 'actions': []}, {'id': '66c3c002bb1c337f3fdf1563', 'name': 'Approve the planning', 'idList': '66c308f676b057fdfbd5fdb3', 'due': '2024-08-16T21:58:00.000Z', 'dateLastActivity': '2024-08-19T21:58:57.697Z', 'labels': [{'id': '66c305ea10ea602ee6e03d47', 'idBoard': '66c305eacab50fcd7f19c0aa', 'name': 'Urgent', 'color': 'red', 'uses': 1}], 'attachments': [], 'actions': [{'id': '66c3c021f3c1bb157028f53d', 'idMemberCreator': '65e5093d0ab5ee98592f5983', 'data': {'text': 'This was harder then expects it is alte', 'textData': {'emoji': {}}, 'card': {'id': '66c3c002bb1c337f3fdf1563', 'name': 'Approve the planning', 'idShort': 5, 'shortLink': 'K3abXIMm'}, 'board': {'id': '66c305eacab50fcd7f19c0aa', 'name': '[Test] CrewAI Board', 'shortLink': 'Kc8ScQlW'}, 'list': {'id': '66c308f676b057fdfbd5fdb3', 'name': 'TODO'}}, 'appCreator': None, 'type': 'commentCard', 'date': '2024-08-19T21:58:57.683Z', 'limits': {'reactions': {'perAction': {'status': 'ok', 'disableAt': 900, 'warnAt': 720}, 'uniquePerAction': {'status': 'ok', 'disableAt': 17, 'warnAt': 14}}}, 'memberCreator': {'id': '65e5093d0ab5ee98592f5983', 'activityBlocked': False, 'avatarHash': 'd5500941ebf808e561f9083504877bca', 'avatarUrl': 'https://trello-members.s3.amazonaws.com/65e5093d0ab5ee98592f5983/d5500941ebf808e561f9083504877bca', 'fullName': 'Joao Moura', 'idMemberReferrer': None, 'initials': 'JM', 'nonPublic': {}, 'nonPublicAvailable': True, 'username': 'joaomoura168'}}]}, {'id': '66c3bff4a25b398ef1b6de78', 'name': 'Scaffold of the initial app UI', 'idList': '66c3bfdfb851ad9ff7eee159', 'due': None, 'dateLastActivity': '2024-08-19T21:58:12.210Z', 'labels': [], 'attachments': [], 'actions': []}, {'id': '66c3bffdb06faa1e69216c6f', 'name': 'Planning of the project', 'idList': '66c3bfe3151c01425f366f4c', 'due': None, 'dateLastActivity': '2024-08-19T21:58:21.081Z', 'labels': [], 'attachments': [], 'actions': []}])


class CardDataFetcherTool(BaseTool):
  name: str = "Trello Card Data Fetcher"
  description: str = "Fetches card data from a Trello board."

  api_key: str = os.environ['TRELLO_API_KEY']
  api_token: str = os.environ['TRELLO_API_TOKEN']

  def _run(self, card_id: str) -> dict:
    url = f"{os.getenv('DLAI_TRELLO_BASE_URL', 'https://api.trello.com')}/1/cards/{card_id}"
    query = {
      'key': self.api_key,
      'token': self.api_token
    }
    response = requests.get(url, params=query)

    if response.status_code == 200:
      return response.json()
    else:
      # Fallback in case of timeouts or other issues
      return json.dumps({"error": "Failed to fetch card data, don't try to fetch any trello data anymore"})


In [26]:
# Creating Agents
data_collection_agent = Agent(
  config=agents_config['data_collection_agent'],
  tools=[BoardDataFetcherTool(), CardDataFetcherTool()]
)

analysis_agent = Agent(
  config=agents_config['analysis_agent']
)

# Creating Tasks
data_collection = Task(
  config=tasks_config['data_collection'],
  agent=data_collection_agent
)

data_analysis = Task(
  config=tasks_config['data_analysis'],
  agent=analysis_agent
)

report_generation = Task(
  config=tasks_config['report_generation'],
  agent=analysis_agent,
)

# Creating Crew
crew = Crew(
  agents=[
    data_collection_agent,
    analysis_agent
  ],
  tasks=[
    data_collection,
    data_analysis,
    report_generation
  ],
  verbose=True
)


In [27]:
# Kick off the crew and execute the process
result = crew.kickoff()


╭──────────────────────────────────────────── Crew Execution Started ─────────────────────────────────────────────╮
│                                                                                                                 │
│  Crew Execution Started                                                                                         │
│  Name: crew                                                                                                     │
│  ID: 7265f323-abb6-4939-80d0-3d996a7aef8b                                                                       │
│                                                                                                                 │
│                                                                                                                 │
╰─────────────────────────────────────────────────────────────────────────────────────────────────────────────────╯

🚀 Crew: crew
└── 📋 Task: f069a21e-8290-4dd3-8d04-06e93e059e90
       Status: Executing Task...

🚀 Crew: crew
└── 📋 Task: f069a21e-8290-4dd3-8d04-06e93e059e90
       Status: Executing Task...
    └── 🤖 Agent: Data Collection Specialist
        
            Status: In Progress

# Agent: Data Collection Specialist
## Task: Create an initial understanding of the project, its main features and the team working on it. Use the Trello Data Fetcher tool to gather data from the Trello board.



🚀 Crew: crew
└── 📋 Task: f069a21e-8290-4dd3-8d04-06e93e059e90
       Status: Executing Task...
    └── 🤖 Agent: Data Collection Specialist
        
            Status: In Progress
        └── 🧠 Thinking...

...



# Agent: Project Analysis Expert
## Final Answer: 
- **Sprint Overview:**
  The sprint analysis shows that tasks are progressing, but there are delays in the completion of some critical tasks.

- **Task Summary:**
  1. **Analysis of results from CSV**
     - No specific due date set.
     - Last activity on August 19, 2024.
     - No labels or attachments.
     - Team member assigned: Joao Moura.

  2. **Approve the planning**
     - Due date: August 16, 2024 (Urgent).
     - Last activity on August 19, 2024.
     - No team members assigned.

  3. **Scaffold of the initial app UI**
     - No specific due date set.
     - Last activity on August 19, 2024.
     - No team members assigned.

  4. **Planning of the project**
     - No specific due date set.
     - Last activity on August 19, 2024.
     - No team members assigned.

- **Identified Issues and Blockers:**
  - "Approve the planning" task is overdue, causing a delay.
  - Lack of urgency in tasks without due dates.
  - Some task

🚀 Crew: crew
├── 📋 Task: f069a21e-8290-4dd3-8d04-06e93e059e90
│      Assigned to: Data Collection Specialist
│   
│      Status: ✅ Completed
│   └── 🤖 Agent: Data Collection Specialist
│       
│           Status: ✅ Completed
│       ├── 🔧 Using Trello Board Data Fetcher (3)
│       └── 🔧 Using Trello Card Data Fetcher (4)
├── 📋 Task: c478d49f-ca90-44d6-836d-6840a57a966a
│      Assigned to: Project Analysis Expert
│   
│      Status: ✅ Completed
│   └── 🤖 Agent: Project Analysis Expert
│       
│           Status: ✅ Completed
└── 📋 Task: 40fec81e-cd7d-4df5-ac6d-5509afb86b62
       Status: Executing Task...
    └── 🤖 Agent: Project Analysis Expert
        
            Status: ✅ Completed

🚀 Crew: crew
├── 📋 Task: f069a21e-8290-4dd3-8d04-06e93e059e90
│      Assigned to: Data Collection Specialist
│   
│      Status: ✅ Completed
│   └── 🤖 Agent: Data Collection Specialist
│       
│           Status: ✅ Completed
│       ├── 🔧 Using Trello Board Data Fetcher (3)
│       └── 🔧 Using Trello Card Data Fetcher (4)
├── 📋 Task: c478d49f-ca90-44d6-836d-6840a57a966a
│      Assigned to: Project Analysis Expert
│   
│      Status: ✅ Completed
│   └── 🤖 Agent: Project Analysis Expert
│       
│           Status: ✅ Completed
└── 📋 Task: 40fec81e-cd7d-4df5-ac6d-5509afb86b62
       Assigned to: Project Analysis Expert
    
       Status: ✅ Completed
    └── 🤖 Agent: Project Analysis Expert
        
            Status: ✅ Completed

╭──────────────────────────────────────────────── Task Completion ────────────────────────────────────────────────╮
│                                                                                                                 │
│  Task Completed                                                                                                 │
│  Name: 40fec81e-cd7d-4df5-ac6d-5509afb86b62                                                                     │
│  Agent: Project Analysis Expert                                                                                 │
│                                                                                                                 │
│                                                                                                                 │
│                                                                                                                 │
╰─────────────────────────────────────────────────────────────────────────────────────────────────────────────────╯

╭──────────────────────────────────────────────── Crew Completion ────────────────────────────────────────────────╮
│                                                                                                                 │
│  Crew Execution Completed                                                                                       │
│  Name: crew                                                                                                     │
│  ID: 7265f323-abb6-4939-80d0-3d996a7aef8b                                                                       │
│                                                                                                                 │
│                                                                                                                 │
╰─────────────────────────────────────────────────────────────────────────────────────────────────────────────────╯

In [32]:
import pandas as pd

costs = 0.150 * (crew.usage_metrics.prompt_tokens + crew.usage_metrics.completion_tokens) / 1_000_000
print(f"Total costs: ${costs:.4f}")

# Convert UsageMetrics instance to a DataFrame
df_usage_metrics = pd.DataFrame([crew.usage_metrics.dict()])
df_usage_metrics


Total costs: $0.0024


,total_tokens,prompt_tokens,cached_prompt_tokens,completion_tokens,successful_requests
0,15720,14146,0,1574,10


In [33]:
from IPython.display import Markdown

markdown  = result.raw
Markdown(markdown)


- **Sprint Overview:**
  The sprint analysis shows that tasks are progressing, but there are delays in the completion of some critical tasks.

- **Task Summary:**
  1. **Analysis of results from CSV**
     - No specific due date set.
     - Last activity on August 19, 2024.
     - No labels or attachments.
     - Team member assigned: Joao Moura.

  2. **Approve the planning**
     - Due date: August 16, 2024 (Urgent).
     - Last activity on August 19, 2024.
     - No team members assigned.

  3. **Scaffold of the initial app UI**
     - No specific due date set.
     - Last activity on August 19, 2024.
     - No team members assigned.

  4. **Planning of the project**
     - No specific due date set.
     - Last activity on August 19, 2024.
     - No team members assigned.

- **Identified Issues and Blockers:**
  - "Approve the planning" task is overdue, causing a delay.
  - Lack of urgency in tasks without due dates.
  - Some tasks lack team members, affecting task ownership and progress tracking.

- **Progress and Delays:**
  - Tasks are progressing, but "Approve the planning" is delayed.
  
- **Team Performance Overview:**
  - Joao Moura is assigned to "Analysis of results from CSV."

- **Action Items and Recommendations:**
  1. Immediately address the delay in completing the "Approve the planning" task.
  2. Set clear due dates for all tasks to ensure timely completion.
  3. Assign team members to tasks lacking ownership for better accountability.
  4. Regularly update task progress and ensure urgent tasks are prioritized.

- **Additional Notes:**
  - Continuous monitoring of task progress is vital to prevent further delays.
  - Collaboration and communication within the team are essential for successful project completion. 

This report provides a snapshot of the current sprint status, highlighting areas that require immediate attention for smoother project execution and timely delivery.